In [15]:
import numpy as np
import json

In [16]:
with open("dataset_infos.json", "r") as json_file:
    datasets = json.load(json_file)

In [17]:
# data_list = datasets
## Create a new dictionary with the desired structure
# dataset_dict = {}
# for data_dict in data_list:
#    if 'dataset' in data_dict:
#        dataset_name = data_dict['dataset']
#        del data_dict['dataset']
#        dataset_dict[dataset_name] = data_dict

In [18]:
# with open("dataset_infos.json", "w") as json_file:
#    json.dump(dataset_dict, json_file)

In [25]:
import os
list_datasets = []
for dataset in os.listdir('datasets/Classical'):
    list_datasets.append(dataset.replace(".npz",""))

In [27]:
def generate_dictionaries_with_dependency(input_dict, dependency_rule):
    import itertools

    # Separate the keys and values, and prepare for combinations excluding the dependent key
    keys, values = zip(*((k, v) for k, v in input_dict.items() if k != "latent_dim"))
    combinations = list(itertools.product(*values))

    # Create dictionaries with combinations of values and add the dependent key-value pair
    result_dicts = []
    for combination in combinations:
        new_dict = dict(zip(keys, combination))
        # Apply the dependency rule to determine the value of the dependent key
        new_dict["latent_dim"] = dependency_rule(new_dict["dataset"])
        result_dicts.append(new_dict)

    return result_dicts

# Define a rule for the dependency of 'key4' on the value of 'key1'
def dependency_rule(value_of_key1: str):
    dataset = value_of_key1.split("_")[1]
    n_features = datasets[dataset]["n_features"]
    if n_features > 30:
        latent_dim = 15
    else:
        latent_dim = n_features / 2
    return int(latent_dim)

In [28]:
predefined_lists_with_dependency = {
    "dataset": list_datasets,
    "kernel": ["rbf"],
    #"batch_size": ["128"],
    #"learning_rate": ["0.01"],
    "latent_dim": [],
    # "loss": ["normal", "loe_hard", "loe_soft"],
    #"loss": ["normal"],
    #"activation": ["relu", "tanh"],
    #"anomaly_type": ["normal","global", "local", "dependency", "clustered"],
    #"noise_type": ["normal","irrelevant_features", "duplicated_anomalies", "annotation_error"],
    "layers": ["5,5", "10,10", "15,15", "10,10,10"],
    #"n_epochs": ["5000"],
}

In [29]:
generated_dictionaries_with_dependency = generate_dictionaries_with_dependency(
    predefined_lists_with_dependency, dependency_rule
)

In [30]:
len(generated_dictionaries_with_dependency)

188

In [31]:
with open("experiment_set_normal_gplvm.json", "w") as json_file:
    json.dump(generated_dictionaries_with_dependency, json_file)

In [9]:
import pandas as pd

df = pd.read_csv(
    "/home/jovyan/work/OE-GPLVM/dataset_mapping.csv",
    names=["dataset", "n_samples", "n_features", "n_anomaly", "pct_anomaly", "domain"],
)

In [12]:
import json

datasets = json.loads(df.to_json(orient="records"))

In [15]:
with open(file_name, "w") as json_file:
    json.dump(data, json_file)

In [16]:
file_name = "dataset_infos.json"

# Use json.dump() with the 'indent' parameter set to 4
with open(file_name, "w") as json_file:
    json.dump(datasets, json_file, indent=4)